In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
pip install -U langchain-community

In [ ]:
pip install chromadb

# Загрузка в chromadb

In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# === Параметры ===
source_dir = "/content/drive/MyDrive/moodle_docs9"         # Папка с .md файлами
persist_dir = "/content/drive/MyDrive/chroma_db_qwen3"              # Папка, куда сохранить ChromaDB
embedding_model_name = "Qwen/Qwen3-Embedding-0.6B"

# === Загрузка всех .md файлов ===
documents = []
for filename in os.listdir(source_dir):
    if filename.endswith(".md"):
        path = os.path.join(source_dir, filename)
        loader = TextLoader(path, encoding="utf-8")
        documents.extend(loader.load())

# === Нарезка на чанки ===
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# === Эмбеддинги ===
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

# === Сохранение в ChromaDB ===
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=persist_dir
)

vectorstore.persist()
print(f"Загружено {len(chunks)} чанков в ChromaDB ({persist_dir})")